In [1]:
from __future__ import annotations
from wildlife_datasets.datasets import CTai
from wildlifetools.wildlife_tools.data import MyWildlifeDataset2
import torchvision.transforms as T
from wildlife_datasets import analysis, datasets, splits

# Load the dataset you want !!
datasets.CTai.get_data('data/CTai')
metadata = CTai('data/CTai')

# # Split the dataset with the same setting of paper !!
splitter = splits.DisjointSetSplit(ratio_class_test = 0.2)
idx_train, idx_test = splitter.split(metadata.df)[0]

splitter_test = splits.ClosedSetSplit(ratio_train = 0.8)
idx_database, idx_query = splitter_test.split(metadata.df.iloc[idx_test])[0]

# Transform and the dataset class
transform = T.Compose([T.Resize([224, 224]), T.ToTensor()])
train_dataset = MyWildlifeDataset2(metadata.df.iloc[idx_train], metadata.root, transform=transform,xy_file="train_data_xy_CTai_3keypoint.pt")
test_database = MyWildlifeDataset2(metadata.df.iloc[idx_database], metadata.root, transform=transform,xy_file="test_database_xy_CTai_3keypoint.pt")
test_query = MyWildlifeDataset2(metadata.df.iloc[idx_query], metadata.root, transform=transform,xy_file="test_query_xy_CTai_3keypoint.pt")

/home/yuhao/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DATASET CTai: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


/home/yuhao/Desktop/Y3/animal_ReID/wildlifetools/wildlife_tools/data/mydataset.py:240: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.xy = torch.load(xy_file)


In [2]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset , batch_size=32, shuffle=True)
test_database_dataloader = DataLoader(test_database , batch_size=16, shuffle=False)
test_query_dataloader = DataLoader(test_query , batch_size=16, shuffle=False)

In [3]:
from transformers import AutoImageProcessor
from modify_vit2 import ViTModel
import torch
from torch import nn
class ViT(nn.Module):
    def __init__(self, num_classes, pool = 'cls'):
        super().__init__()

        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        # self.basemodel = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
        # self.basemodel = ViTModel.from_pretrained("google/vit-huge-patch14-224-in21k")
        self.basemodel = ViTModel.from_pretrained("google/vit-large-patch16-224-in21k")
        
        
        
        self.pool = pool
        # self.mlp_head = nn.Linear(768, num_classes)
        # self.mlp_head = nn.Linear(1280, num_classes)
        self.mlp_head = nn.Linear(1024, num_classes)
    
    

    def forward(self, x, pos):
        x = self.basemodel(x, kp_pos = pos).last_hidden_state
        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        return x, self.mlp_head(x)

2024-11-27 18:48:07.978786: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 18:48:08.074968: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yuhao/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda-12.0/lib64:
2024-11-27 18:48:08.074989: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-27 18:48:08.593296: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic librar

In [4]:

model = ViT(train_dataset.num_classes, pool = 'cls')
model = model.to('cuda')
for name, param in model.named_parameters():
    # print(name)
    # if '10' in name or '11' in name:
    if '0' in name:
        # print(name)
    
        param.requires_grad = True
    # elif 'mlp' in name:
    elif 'mlp' in name or 'kp_position_embedding' in name:
        print(name)
       
        param.requires_grad = True
    else:
        param.requires_grad = False
        
import torch.optim as optim
import torch.nn as nn
from wildlife_tools.train import ArcFaceLoss,TripletLoss
# criterion_mat = ArcFaceLoss(num_classes=train_dataset.num_classes, embedding_size=768, margin=0.5, scale=64)
# criterion_mat = ArcFaceLoss(num_classes=train_dataset.num_classes, embedding_size=1280, margin=0.5, scale=64)
criterion_mat = ArcFaceLoss(num_classes=train_dataset.num_classes, embedding_size=1024, margin=0.5, scale=16)


criterion_cls = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['embeddings.kp_position_embeddings', 'embeddings.kp_position_embeddings2.bias', 'embeddings.kp_position_embeddings2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


basemodel.embeddings.kp_position_embeddings
basemodel.embeddings.kp_position_embeddings2.weight
basemodel.embeddings.kp_position_embeddings2.bias
mlp_head.weight
mlp_head.bias


In [5]:
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
import numpy as np
from torch.utils.data import DataLoader

def eva():
    model.eval()
    database = []
    query = []
    label_map_train = []
    label_map_test = []
    for i, data in enumerate(test_database_dataloader, 0):
        inputs, pos, labels = data
        inputs, pos, labels = inputs.to('cuda'), pos.to('cuda'), labels.to('cuda')
        # outputs_emb, _ = net(inputs[:,:196,:])
        outputs_emb, _ = model(inputs, pos)
    
        database.append(outputs_emb.cpu().detach())
        label_map_train = np.append(labels.cpu().detach().numpy(),label_map_train)
    database = torch.cat(database).numpy()
    # print(database.shape)
    for i, data in enumerate(test_query_dataloader, 0):
        inputs, pos, labels = data
        inputs, pos, labels = inputs.to('cuda'), pos.to('cuda'), labels.to('cuda')
        # outputs_emb, _ = net(inputs[:,:196,:])
        outputs_emb, _ = model(inputs, pos)
    
        query.append(outputs_emb.cpu().detach())
        label_map_test = np.append(labels.cpu().detach().numpy(),label_map_test)
    query = torch.cat(query).numpy()

    similarity = CosineSimilarity()
    sim = similarity(query, database)['cosine']

    classifier = KnnClassifier(k=1,database_labels=test_database.labels_string)
    preds = classifier(sim)
    acc = sum(preds == test_query.labels_string) / len(label_map_test)
    print('\n Accuracy: ', acc)

/home/yuhao/.local/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [6]:
from tqdm import tqdm
model = model.train()

epochs = 100
for epoch in range(epochs): 
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, pos, labels = data
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        pos = pos.to('cuda')
        
        optimizer.zero_grad()
        outputs_emb, outputs_class  = model(inputs,pos)
        loss = 1*  criterion_mat(outputs_emb, labels) + 0.2*criterion_cls(outputs_class, labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()
        
    print(f'[{epoch + 1}] loss: {running_loss:.3f}')
    eva()
    running_loss = 0.0
    
    
 
print('Finished Training')

[1] loss: 1399.019

 Accuracy:  0.7345971563981043
[2] loss: 1346.687

 Accuracy:  0.7725118483412322
[3] loss: 1295.986

 Accuracy:  0.8246445497630331
[4] loss: 1229.317

 Accuracy:  0.8104265402843602
[5] loss: 1140.696

 Accuracy:  0.8151658767772512
[6] loss: 1065.326

 Accuracy:  0.8578199052132701
[7] loss: 991.820

 Accuracy:  0.8578199052132701
[8] loss: 915.914

 Accuracy:  0.8672985781990521
[9] loss: 849.974

 Accuracy:  0.8625592417061612
[10] loss: 778.620

 Accuracy:  0.8767772511848341
[11] loss: 706.197

 Accuracy:  0.8483412322274881
[12] loss: 638.275

 Accuracy:  0.8483412322274881
[13] loss: 568.907

 Accuracy:  0.8625592417061612
[14] loss: 512.470

 Accuracy:  0.8625592417061612
[15] loss: 459.382

 Accuracy:  0.8530805687203792
[16] loss: 414.730

 Accuracy:  0.8388625592417062
[17] loss: 362.477

 Accuracy:  0.8151658767772512
[18] loss: 326.315

 Accuracy:  0.8293838862559242
[19] loss: 289.764

 Accuracy:  0.8436018957345972
[20] loss: 258.893

 Accuracy:  0.